### TP3 - 2

2. O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.


<img src="inversores.png" alt="autómato"/>


   1. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   2. Cada inversor é regido pelas seguintes transformações 
  
     $$ \mathbf{invert}\mathtt(in,out) $$
     $$x \gets \mathsf{read}(\mathtt{in})$$
     $$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$             
     $$\mathsf{write}(\mathtt{out},s)$$
     
        iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
        
        iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
        
        iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.


1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
    
2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import itertools 
from random import random, choice

### Função genState bla bla bla

In [2]:
# vars = ['pc', s']
def genState(vars,s,i):
    state = {}
    state['pc'] = Symbol('pc'+'_'+s+str(i), INT)
    for j in range(1, 5):
        for v in vars[1:]:
            if v not in state:
                state[v] = {}
            
            state[v][j] = Symbol(v+str(j)+'_'+s+str(i))
            
    #print(state)
    
            
    return state

#### Função init(state):

    state - Dicionário de variáveis de estado


   A função `init` tem como objetivo devolver um predicado do Solver que testa se é um possível estado inicial do programa, através do `state`, um dicionário de variáveis.

In [3]:
def init(state):
    #random_state = (Bool(choice([True, False])),Bool(choice([True, False])),Bool(choice([True, False])),Bool(choice([True, False]))) 
    
    random_state = (Bool(True), Bool(True), Bool(True), Bool(True))

    s_list = []
    for v in state['s']:
        s_list.append( Iff(state['s'][v], random_state[v-1]) )
    
    return And(
        Equals(state['pc'], Int(1)),
        And(s_list)
    )

#### Função trans(curr, prox):

    curr - Estado das variáveis no momento atual
    prox - Estado das variáveis no momento da próxima iteração
    
   A função `trans` tem como objetivo devolver um predicado do Solver, através dos três estados disponíveis, que teste se é possível transitar entre os estados possíveis.

In [4]:
def error(state):
    return Or( 
        Iff(Or([state['s'][i] for i in range(1,5)]), Bool(False))
    )


def trans(curr, prox):
    zero_state = Iff(Or([curr['s'][i] for i in range(1,5)]), Bool(False))
    
    # prox_x = prox_s
    # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])  
    t1_2 = And(
        Equals(curr['pc'], Int(1)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], Int(2))),
            And(zero_state, Equals(prox['pc'], Int(5)))
        ),
        
        # o x é o s do inversor anterior
        # prox_s = Not(curr[x])  V Xor(curr[s], curr[x])
        Iff(prox['s'][1], Or(Not(curr['s'][4]), Xor(curr['s'][1], curr['s'][4]) )), 
        
        And([ Iff(prox['s'][i], curr['s'][i]) for i in range(2,5)])
        
    )  
    
    t2_3 = And(
        Equals(curr['pc'], Int(2)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], Int(3))),
            And(zero_state, Equals(prox['pc'], Int(5)))
        ),
        
        Iff(prox['s'][2], Or(Not(curr['s'][1]), Xor(curr['s'][2], curr['s'][1]) )),
        
        Iff(prox['s'][1], curr['s'][1]),
        And([Iff(prox['s'][i], curr['s'][i]) for i in range(3,5)]) 
    )
    
    t3_4 = And(
        Equals(curr['pc'], Int(3)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], Int(4))),
            And(zero_state, Equals(prox['pc'], Int(5)))
        ),
        
        Iff(prox['s'][3], Or(Not(curr['s'][2]), Xor(curr['s'][3], curr['s'][2]) )),
        
        And([Iff(prox['s'][i], curr['s'][i]) for i in range(1,3)]),
        Iff(prox['s'][4], curr['s'][4])
    )
    
    t4_1 = And(
        Equals(curr['pc'], Int(4)),
            
        Or(
            And(Not(zero_state), Equals(prox['pc'], Int(1))),
            And(zero_state, Equals(prox['pc'], Int(5)))
        ),
        
        Iff(prox['s'][4], Or(Not(curr['s'][3]), Xor(curr['s'][4], curr['s'][3]) )),
        
        And([Iff(prox['s'][i], curr['s'][i]) for i in range(1,4)])
    )
    
    
    error = And(
        Equals(curr['pc'], Int(5)),
        Equals(prox['pc'], curr['pc']),
        zero_state,
        And( [Iff(prox['s'][i], curr['s'][i]) for i in range(1,5)])
    )
    
    return Or(t1_2, t2_3, t3_4, t4_1, error)
    

#### Função gera_traco(declare, init, trans, k, n, a, b)

    declare - Variáveis de estado
    init - Condições para o estado inicial
    trans - Função transição
    k - Valor do traço


A função `gera_traco` tem como objetivo imprimir o valor das variáveis à medida que vão percorrendo os estados, através das variáveis do estado e de um predicado que testa se um estado é inicial.

In [5]:
def gera_traco(vars, init, trans, error, k):
    with Solver(name="z3") as s:
    
        X = [genState(vars,'X',i) for i in range(k+1)]
        #print(X)
        I = init(X[0])
        print(I)
        Tks = [ trans(X[i],X[i+1]) for i in range(k) ]
        #print(Tks)
    
        if s.solve([I,And(Tks)]):
            for i in range(k):
                print("Passo:",i)
                for v in X[i]:
                    if v == 'pc':
                        print("Estado: ",s.get_value(X[i][v]))
                    else:
                        for k in range(1,5):
                            print("          ",v,'=',s.get_value(X[i][v][k]))
                print("----------------")
        else:
            print("unsat")
        
        
gera_traco(['pc', 's'], init, trans, error, 20)

((pc_X0 = 1) & ((s1_X0 <-> True) & (s2_X0 <-> True) & (s3_X0 <-> True) & (s4_X0 <-> True)))
Passo: 0
Estado:  1
           s = True
           s = True
           s = True
           s = True
----------------
Passo: 1
Estado:  2
           s = False
           s = True
           s = True
           s = True
----------------
Passo: 2
Estado:  3
           s = False
           s = True
           s = True
           s = True
----------------
Passo: 3
Estado:  4
           s = False
           s = True
           s = False
           s = True
----------------
Passo: 4
Estado:  1
           s = False
           s = True
           s = False
           s = True
----------------
Passo: 5
Estado:  2
           s = True
           s = True
           s = False
           s = True
----------------
Passo: 6
Estado:  3
           s = True
           s = False
           s = False
           s = True
----------------
Passo: 7
Estado:  4
           s = True
           s = False
           s = True